In [1]:
import time
from dataclasses import dataclass
import json
import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import  GemmaTokenizerFast, BitsAndBytesConfig, Gemma2ForCausalLM
from transformers.data.data_collator import pad_without_fast_tokenizer_warning

from transformers import  GemmaTokenizerFast, BitsAndBytesConfig, Gemma2ForCausalLM

/home/chenning/miniconda3/envs/llm_v1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_path = '../datas/train.csv'
test_path = '../datas/test.csv'
sample_path = '../datas/sample_submission.csv'
mapping_path = '../datas/misconception_mapping.csv'

m_map = pd.read_csv(mapping_path)
m_map.head()

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,1,Uses dividing fractions method for multiplying...
2,2,Believes there are 100 degrees in a full turn
3,3,Thinks a quadratic without a non variable term...
4,4,Believes addition of terms and powers of terms...


In [29]:
train_path = '../datas/data/train_data.json'
dev_path = '../datas/data/dev_data.json'
all_path = '../datas/data/all_data.json'

train_df = pd.read_json(train_path)
dev_df = pd.read_json(dev_path)
all_df = pd.read_json(all_path)
all_df = all_df[all_df.MisconceptionId != -1].reset_index(drop=True)
all_df['QuestionId'] = all_df['QuestionId'].astype(str)
all_df['QuestionId_Answer'] = all_df['QuestionId'] + "_" + all_df['InCorrectAnswer']


print(len(train_df), len(dev_df), len(all_df))
train_df.head()

3770 600 4370


,QuestionId_Answer,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName
0,0_D,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,A,\( 3 \times(2+4)-5 \),D,Does not need brackets,1672,"Confuses the order of operations, believes add..."
1,1_A,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,A,\( m+1 \),2142,Does not know that to factorise a quadratic ex...
2,1_B,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,B,\( m+2 \),143,Thinks that when you cancel identical terms fr...
3,1_C,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,C,\( m-1 \),2142,Does not know that to factorise a quadratic ex...
4,2_A,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,B,Only\nKatie,A,Only\nTom,1287,Believes if you changed all values by the same...


In [8]:
data_path = './data/stage1/data_stage1.json'
data_df = pd.read_json(data_path)
# data_df['QuestionId'] = data_df['QuestionId'].astype(str)
# data_df['QuestionId_Answer'] = data_df['QuestionId'] + "_" + data_df['InCorrectAnswer']
# data_df = data_df[data_df.MisconceptionId != -1].reset_index(drop=True)
print(len(data_df))
data_df.head()

4370


,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName,QuestionId_Answer,MisconceptionId_y,MisconceptionText
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,A,\( 3 \times(2+4)-5 \),D,Does not need brackets,1672,"Confuses the order of operations, believes add...",0_D,"[1005, 1507, 2488, 2586, 2221, 1999, 1672, 134...",1. Carries out operations from left to right r...
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,A,\( m+1 \),2142,Does not know that to factorise a quadratic ex...,1_A,"[891, 1755, 59, 167, 1554, 1593, 1610, 2398, 1...",1. Incorrectly cancels what they believe is a ...
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,B,\( m+2 \),143,Thinks that when you cancel identical terms fr...,1_B,"[1755, 891, 59, 167, 1554, 1593, 1610, 2398, 2...",1. Incorrectly factorises a quadratic\n2. Inco...
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,C,\( m-1 \),2142,Does not know that to factorise a quadratic ex...,1_C,"[143, 891, 2398, 885, 59, 1610, 1554, 1593, 15...",1. Thinks that when you cancel identical terms...
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,B,Only\nKatie,A,Only\nTom,1287,Believes if you changed all values by the same...,2_A,"[1073, 1287, 2471, 1677, 2439, 397, 1349, 1059...",1. Believes if you add the same value to all n...


In [5]:
# 临时数据
import os
tmp_path = './tmp_stage2_data'
os.makedirs(tmp_path, exist_ok=True)

# prompt_data_path = os.path.join(tmp_path, "prompt_data.parquet")

In [9]:

def stage2_prompt_1(data):
    def apply_template(row):
        prompt_text_1 = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>"
        prompt1 = 'Here are students answer Diagnostic Questions (DQs), which are choice questions featuring one correct answer and one incorrect answers, known as distractors. Each question targets a specific construct (also referred to as a skill), representing the most granular level of knowledge relevant to the question.\nYour task: According to the following Incorrect answer, the most likely 25 misconceptions, perhaps the real misconception is among them. Identify the misconception behind Incorrect Answer. \n#{}\n\n'
        prompt1 = prompt1.format('#most likely 25 misconceptions:\n' + row['MisconceptionText'])
        
        # prompt1 = 'Here are students answer Diagnostic Questions (DQs), which are choice questions featuring one correct answer and one incorrect answers, known as distractors. Each question targets a specific construct (also referred to as a skill), representing the most granular level of knowledge relevant to the question.\nYour task: Identify the misconception behind Incorrect Answer.\n\n'
        prompt2  = """#Construct Name: {ConstructName}\n#Question: {Question}\n#Correct Answer: {CorrectAnswer}\n#Incorrect Answer: {IncorrectAnswer}"""
        prompt = prompt2.format(
                 ConstructName=row["ConstructName"],
                 Question=row["QuestionText"],
                 CorrectAnswer=row['CorrectAnswerText'],
                 IncorrectAnswer=row['InCorrectAnswerText'])
        prompt_response = prompt_text_1.format(prompt1 + prompt + "\n" + prompt + "\n" + prompt)
        return prompt_response

    def apply_template2(row):
        prompt2  = """#Construct Name: {ConstructName}\n#Question: {Question}\n#Correct Answer: {CorrectAnswer}\n#Incorrect Answer: {IncorrectAnswer}"""
        prompt = prompt2.format(
                 ConstructName=row["ConstructName"],
                 Question=row["QuestionText"],
                 CorrectAnswer=row['CorrectAnswerText'],
                 IncorrectAnswer=row['InCorrectAnswerText'])
        return prompt

    
    
    data['prompt_response'] = data.apply(apply_template, axis=1)
    data['prompt'] = data.apply(apply_template2, axis=1)
    
    return data
    
    
data = stage2_prompt_1(data_df)
data.to_parquet('./tmp_stage2_data/prompt_data.parquet', index=False)
print(data.prompt_response.values[0])
print(len(data))


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Here are students answer Diagnostic Questions (DQs), which are choice questions featuring one correct answer and one incorrect answers, known as distractors. Each question targets a specific construct (also referred to as a skill), representing the most granular level of knowledge relevant to the question.
Your task: According to the following Incorrect answer, the most likely 25 misconceptions, perhaps the real misconception is among them. Identify the misconception behind Incorrect Answer. 
##most likely 25 misconceptions:
1. Carries out operations from left to right regardless of priority order, unless brackets are used
2. Carries out operations from left to right regardless of priority order
3. Answers order of operations questions with brackets as if the brackets are not there
4. Misunderstands order of operations in algebraic expressions
5. Has made an order of operations mistake when writing a formula from a given situat

In [12]:
%%time
!/home/chenning/miniconda3/envs/llm_v2/bin/python run_vllm.py\
    --gpu 2 \
    --input_file ./tmp_stage2_data/prompt_data.parquet \
    --output_file ./tmp_stage2_data/llm_submission_stage2.parquet \
    --tensor_parallel_size 1 \
    --lora_dir ../output/autumn-bird-58/checkpoint-1175 \






lora path: ../output/autumn-bird-58/checkpoint-1175
llm path: /home/chenning/opensource_models/llama/Meta-Llama-3.1-8B-Instruct
Namespace(model_dir='/home/chenning/opensource_models/llama/Meta-Llama-3.1-8B-Instruct', lora_dir='../output/autumn-bird-58/checkpoint-1175', input_file='./tmp_stage2_data/prompt_data.parquet', output_file='./tmp_stage2_data/llm_submission_stage2.parquet', batch_size=4, gpu='2', max_length=2200, max_tokens=1024, seed=777, num_responses=1, spread_max_length=False, temperature=0, top_p=0.9, tensor_parallel_size=1)
WARNING 10-23 08:55:57 config.py:1563] Casting torch.bfloat16 to torch.float16.
INFO 10-23 08:55:57 llm_engine.py:184] Initializing an LLM engine (v0.5.5) with config: model='/home/chenning/opensource_models/llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='/home/chenning/opensource_models/llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_

In [35]:
## 构建召回数据


llm_output = pd.read_parquet("./tmp_stage2_data/llm_submission_stage2.parquet")
for idx, row in llm_output[-5:].iterrows():
    print(row.text)
    print("==="*6)


tmp = []
for i in range(len(llm_output)):
    # text = f"ConstructName:{llm_output.loc[i, 'ConstructName']}\nQuestionText:{llm_output.loc[i, 'QuestionText']}\nCorrectAnswerText:{llm_output.loc[i, 'CorrectAnswerText']}\nInCorrectAnswerText:{llm_output.loc[i, 'InCorrectAnswerText']}\nmisconception:{llm_output.loc[i, 'text']}"
    text = f"ConstructName:{llm_output.loc[i, 'ConstructName']}\nQuestionText:{llm_output.loc[i, 'QuestionText']}\nInCorrectAnswerText:{llm_output.loc[i, 'InCorrectAnswerText']}\nmisconception:{llm_output.loc[i, 'text']}"
    
    tmp.append(text)
    
llm_output['text'] = tmp
print(llm_output['text'].values[0])
print(len(llm_output))
llm_output.to_parquet("./tmp_stage2_data/recall_data_2.parquet", index=False)
llm_output.head()

Thinks congruent shapes are the same shape but can be different sizes
Thinks congruent shapes are the same shape but can be different sizes
Forgets to give the centre of rotation
Forgets to give the centre of rotation
Forgets to give the centre of rotation
ConstructName:Use the order of operations to carry out calculations involving powers
QuestionText:\[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ?
InCorrectAnswerText:Does not need brackets
misconception:Carries out operations from left to right regardless of priority order
4370


,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName,QuestionId_Answer,MisconceptionId_y,MisconceptionText,prompt_response,prompt,text
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,A,\( 3 \times(2+4)-5 \),D,Does not need brackets,1672,"Confuses the order of operations, believes add...",0_D,"[1005, 1507, 2488, 2586, 2221, 1999, 1672, 134...",1. Carries out operations from left to right r...,<|begin_of_text|><|start_header_id|>user<|end_...,#Construct Name: Use the order of operations t...,ConstructName:Use the order of operations to c...
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,A,\( m+1 \),2142,Does not know that to factorise a quadratic ex...,1_A,"[891, 1755, 59, 167, 1554, 1593, 1610, 2398, 1...",1. Incorrectly cancels what they believe is a ...,<|begin_of_text|><|start_header_id|>user<|end_...,#Construct Name: Simplify an algebraic fractio...,ConstructName:Simplify an algebraic fraction b...
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,B,\( m+2 \),143,Thinks that when you cancel identical terms fr...,1_B,"[1755, 891, 59, 167, 1554, 1593, 1610, 2398, 2...",1. Incorrectly factorises a quadratic\n2. Inco...,<|begin_of_text|><|start_header_id|>user<|end_...,#Construct Name: Simplify an algebraic fractio...,ConstructName:Simplify an algebraic fraction b...
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,C,\( m-1 \),2142,Does not know that to factorise a quadratic ex...,1_C,"[143, 891, 2398, 885, 59, 1610, 1554, 1593, 15...",1. Thinks that when you cancel identical terms...,<|begin_of_text|><|start_header_id|>user<|end_...,#Construct Name: Simplify an algebraic fractio...,ConstructName:Simplify an algebraic fraction b...
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,B,Only\nKatie,A,Only\nTom,1287,Believes if you changed all values by the same...,2_A,"[1073, 1287, 2471, 1677, 2439, 397, 1349, 1059...",1. Believes if you add the same value to all n...,<|begin_of_text|><|start_header_id|>user<|end_...,#Construct Name: Calculate the range from a li...,ConstructName:Calculate the range from a list ...


In [36]:
%%time
!/home/chenning/miniconda3/envs/llm_v2/bin/python run_similarity_search_nv.py\
    --gpu 3,4 \
    --input_file ./tmp_stage2_data/recall_data_2.parquet \
    --output_file ./tmp_stage2_data/submission_stage2.csv \
    --batch_size 5 \
    --recall_num 3000 \






/home/chenning/opensource_models/nvidia/NV-Embed-v2
Namespace(model_dir='/home/chenning/opensource_models/nvidia/NV-Embed-v2', input_file='./tmp_stage2_data/recall_data_2.parquet', output_file='./tmp_stage2_data/submission_stage2.csv', batch_size=5, gpu='3,4', max_length=512, seed=777, recall_num=3000, num_beam=1, spread_max_length=False, temp=1.03, use_lora=False, lora_path='../recall_model_save/nv_round1_qlora_rerun/epoch_19_model/adapter.bin')
  0%|                                                   | 0/874 [00:00<?, ?it/s]/home/chenning/.cache/huggingface/modules/transformers_modules/NV-Embed-v2/modeling_nvembed.py:350: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),
/home/chenning/miniconda3/envs/llm_v2/lib/python3.11/contextlib.py:105: Futu

In [37]:
recall_res = pd.read_csv("./tmp_stage2_data/submission_stage2.csv")[['QuestionId_Answer', 'MisconceptionId']]
print(recall_res.shape)

llm_res = pd.read_parquet("./tmp_stage2_data/llm_submission_stage2.parquet")[['QuestionId_Answer', 'text']]
print(llm_res.shape)
data_df2 = llm_res.merge(recall_res, on='QuestionId_Answer', how='left')
data_df2 = data_df2.rename(columns={"MisconceptionId":"recallMisconceptionId"})
data_df2 = all_df.merge(data_df2, on='QuestionId_Answer', how='left')

data_df2.head()

(4370, 2)
(4370, 2)


,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName,QuestionId_Answer,text,recallMisconceptionId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,A,\( 3 \times(2+4)-5 \),D,Does not need brackets,1672,"Confuses the order of operations, believes add...",0_D,Carries out operations from left to right rega...,1005 1507 2488 2586 2221 1999 1672 1345 706 25...
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,A,\( m+1 \),2142,Does not know that to factorise a quadratic ex...,1_A,Incorrectly cancels what they believe is a fac...,891 59 2398 1593 885 1610 1540 1554 1755 1262 ...
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,B,\( m+2 \),143,Thinks that when you cancel identical terms fr...,1_B,Incorrectly factorises a quadratic,1755 167 265 2068 59 2142 3 1593 891 1554 838 ...
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,C,\( m-1 \),2142,Does not know that to factorise a quadratic ex...,1_C,Thinks that when you cancel identical terms fr...,143 891 2398 885 59 1610 1554 1593 1540 715 60...
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,B,Only\nKatie,A,Only\nTom,1287,Believes if you changed all values by the same...,2_A,Believes if you changed all values by the same...,1287 1073 1677 2471 2439 397 1059 1349 1177 69...


In [38]:
save_path = './data/stage2'
os.makedirs(save_path, exist_ok=True)


def save_json(json_path, json_df):
    df_str = json_df.to_json()
    print(df_str[0])
    with open(json_path, 'w') as f:
        f.write(df_str)
    print(f"save {json_path} finish")


save_json(os.path.join(save_path, "data_stage2.json"), data_df2)


dev_id = list(dev_df['QuestionId_Answer'].unique())

dev_df2 = data_df2[data_df2.QuestionId_Answer.isin(dev_id)].reset_index(drop=True)
train_df2 = data_df2[~data_df2.QuestionId_Answer.isin(dev_id)].reset_index(drop=True)
print(train_df2.shape, dev_df2.shape)
save_json(os.path.join(save_path, "dev_stage2.json"), dev_df2)

save_json(os.path.join(save_path, "train_stage2.json"), train_df2)



{
save ./data/stage2/data_stage2.json finish
(3770, 15) (600, 15)
{
save ./data/stage2/dev_stage2.json finish
{
save ./data/stage2/train_stage2.json finish


In [39]:
dev_df2.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName,QuestionId_Answer,text,recallMisconceptionId
0,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,Convert this percentage to a fraction\n\( 62 \...,B,\( \frac{31}{50} \),A,\( \frac{62}{10} \),329,Does not understand a percentage is out of 100,6_A,Thinks that a percentage can be converted to a...,1955 847 318 122 1914 292 2361 1822 1786 1270 ...
1,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,Convert this percentage to a fraction\n\( 62 \...,B,\( \frac{31}{50} \),C,\( \frac{6}{2} \),847,Thinks they can convert a 2 digit percentage t...,6_C,Thinks they can convert a 2 digit percentage t...,847 122 318 1955 1786 292 1221 1822 2368 2361 ...
2,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,Convert this percentage to a fraction\n\( 62 \...,B,\( \frac{31}{50} \),D,None of these,329,Does not understand a percentage is out of 100,6_D,Believes percentages cannot be converted into ...,804 847 122 1955 2361 318 145 1822 1396 1786 1...
3,7,314,Divide decimals by 10,224,Multiplying and Dividing with Decimals,\( 43.2 \div 10= \),A,\( 4.32 \),B,\( 0.432 \),2123,Divided by 100 rather than 10,7_B,"When dividing a decimal by a multiple of 10, j...",2133 1983 544 447 2272 2200 1619 2002 619 2525...
4,7,314,Divide decimals by 10,224,Multiplying and Dividing with Decimals,\( 43.2 \div 10= \),A,\( 4.32 \),C,\( 33.2 \),2273,Subtracts instead of divides,7_C,"When dividing a decimal by a multiple of 10, j...",2133 1983 544 2272 447 2200 1619 619 2002 2525...


In [40]:
# 验证

import pandas as pd
pred_labels = list(dev_df2["recallMisconceptionId"].apply(lambda x: [int(y) for y in x.strip('[]').split(' ')]).values)
true_labels = list(dev_df2["MisconceptionId"].values)

def mean_average_precision(true_labels, predicted_labels_list, k=25):

    average_precisions = []
    tmp = []
    for true_label, predicted_labels in zip(true_labels, predicted_labels_list):
        if true_label in predicted_labels[:k]:
            idx = predicted_labels[:k].index(true_label)
            average_precisions.append(1/(idx + 1))
            tmp.append(1)
        else:
            average_precisions.append(0)
            tmp.append(0)
            
    print(sum(tmp) / len(tmp) if tmp else 0)
    return sum(average_precisions) / len(average_precisions) if average_precisions else 0
    
score = mean_average_precision(true_labels,pred_labels)
print(score)

0.7333333333333333
0.37827187186028544


# 构建训练召回数据集验证集不用构建


In [43]:
# train_df = pd.read_json('../datas/data/train_data.json')
train_df = pd.read_json("./data/stage2/train_stage2.json")
# train_df = train_df.merge(train_sub_df, on='QuestionId_Answer', how='left')
train_df.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName,QuestionId_Answer,text,recallMisconceptionId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,A,\( 3 \times(2+4)-5 \),D,Does not need brackets,1672,"Confuses the order of operations, believes add...",0_D,Carries out operations from left to right rega...,1005 1507 2488 2586 2221 1999 1672 1345 706 25...
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,A,\( m+1 \),2142,Does not know that to factorise a quadratic ex...,1_A,Incorrectly cancels what they believe is a fac...,891 59 2398 1593 885 1610 1540 1554 1755 1262 ...
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,B,\( m+2 \),143,Thinks that when you cancel identical terms fr...,1_B,Incorrectly factorises a quadratic,1755 167 265 2068 59 2142 3 1593 891 1554 838 ...
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,C,\( m-1 \),2142,Does not know that to factorise a quadratic ex...,1_C,Thinks that when you cancel identical terms fr...,143 891 2398 885 59 1610 1554 1593 1540 715 60...
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,B,Only\nKatie,A,Only\nTom,1287,Believes if you changed all values by the same...,2_A,Believes if you changed all values by the same...,1287 1073 1677 2471 2439 397 1059 1349 1177 69...


In [46]:
def get_neg(label, preds, k=5):
    preds_list = preds.split()
    label = int(label)
    
    samples = []
    for neg in preds_list:
        neg = int(neg)
        if neg == label:continue
        samples.append(mis_concepts[neg])
    return samples[0:k]
mis_concepts = m_map['MisconceptionName'].tolist()



In [47]:
total_examples = []
for i in range(len(train_df)):
    tmp = {}
    tmp['query'] = f"ConstructName:{train_df.loc[i, 'ConstructName']}\nQuestionText:{train_df.loc[i, 'QuestionText']}\nCorrectAnswerText:{train_df.loc[i, 'CorrectAnswerText']}\nInCorrectAnswerText:{train_df.loc[i, 'InCorrectAnswerText']}"
    tmp['pos'] = [train_df.loc[i, 'MisconceptionName']]
    tmp['neg'] = get_neg(int(train_df.loc[i, "MisconceptionId"]), train_df.loc[i, "recallMisconceptionId"], 50)
    total_examples.append(tmp)
s = set()
for k in total_examples:
    s |= set(k['neg'])

# assert len(s) == len(mis_concepts)

In [48]:
import json
filename = '../llm_recall/datas/stage2_recall/finetune_train.jsonl'
with open(filename, 'w', encoding='utf-8') as f:
    for entry in total_examples:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')  